# Tutorial 4 Solution

In [1]:
import numpy as np

In [2]:
# matrix representation of some basic quantum gates
H = 1/np.sqrt(2) * np.array([[1, 1], [1,-1]])
X = np.array([[0, 1], [1, 0]])
Z = np.array([[1, 0], [0,-1]])
CNOT  = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])
rCNOT = np.array([[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0]])

In [3]:
# "reversed" CNOT gate: second qubit is control and first qubit is target
rCNOT

array([[1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 1, 0, 0]])

### T4(b) -  There is a Hadamard acting on the first qubit, and identities on the remaining three.

In [4]:
H1 = np.kron(H, np.eye(8))
print("Dimensions:", H1.shape)
print(H1)

Dimensions: (16, 16)
[[ 0.70710678  0.          0.          0.          0.          0.
   0.          0.          0.70710678  0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.70710678  0.          0.          0.          0.
   0.          0.          0.          0.70710678  0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.70710678  0.          0.          0.
   0.          0.          0.          0.          0.70710678  0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.70710678  0.          0.
   0.          0.          0.          0.          0.          0.70710678
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.70710678  0.
   0.          0.          0.          0.          0.          0.
   0.70710678  0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.   

### Quick test for T4(c)

The next cell will result in a memory error. Some computers terminate the process early, some may try to allocate as much memory as possible before failing. If yours is the second type, stop the process manually.

In [5]:
# np.kron(H, np.eye(2**19))

### T4(d & e) Matrix-free implemention of single- and two-qubit gates

In [6]:
def apply_hadamard(psi, i):
    """
    Applies the Hadamard gate to qubit `i` (counting from 0) of the input quantum statevector `psi`.
    """
    N = int(np.log2(len(psi)))
    psi = np.reshape(psi, (2**i, 2, 2**(N-i-1)))
    psi_out = psi.copy()
    psi_out[:, 0, :] = (psi[:, 0, :] + psi[:, 1, :]) / np.sqrt(2)
    psi_out[:, 1, :] = (psi[:, 0, :] - psi[:, 1, :]) / np.sqrt(2)
    return psi_out.reshape(-1)

In [7]:
def crandn(size):
    """
    Draw random samples from the standard complex normal (Gaussian) distribution.
    """
    # 1/sqrt(2) is a normalization factor
    return (np.random.normal(size=size) + 1j*np.random.normal(size=size)) / np.sqrt(2)

In [8]:
# test using a random input state
ψtest = crandn(2**4)
ψtest /= np.linalg.norm(ψtest)

print("Apply Hadamard on first qubit correct?:", np.allclose(H1 @ ψtest, apply_hadamard(ψtest, 0)))

Apply Hadamard on first qubit correct?: True


In [9]:
def apply_X(psi, i):
    """
    Applies X to qubit `i` (counting from 0) of input quantum statevector `psi`.
    """
    N = int(np.log2(len(psi)))
    psi = np.reshape(psi, (2**i, 2, 2**(N-i-1)))
    psi_out = psi.copy()
    psi_out[:, 0, :] = psi[:, 1, :]
    psi_out[:, 1, :] = psi[:, 0, :]
    return psi_out.reshape(-1)

In [10]:
print("Apply X on third qubit correct?:", np.allclose(np.kron(np.eye(4), np.kron(X, np.eye(2))) @ ψtest, apply_X(ψtest, 2)))

Apply X on third qubit correct?: True


In [11]:
def apply_Z(psi, i):
    """
    Applies Z to qubit `i` (counting from 0) of input quantum statevector `psi`.
    """
    N = int(np.log2(len(psi)))
    psi = np.reshape(psi, (2**i, 2, 2**(N-i-1)))
    psi_out = psi.copy()
    psi_out[:, 1, :] = -psi[:, 1, :]
    return psi_out.reshape(-1)

In [12]:
print("Apply Z on fourth qubit correct?:", np.allclose(np.kron(np.eye(8), Z) @ ψtest, apply_Z(ψtest, 3)))

Apply Z on fourth qubit correct?: True


In [13]:
def apply_CNOT(psi, i_control, i_target):
    """
    Apply CNOT for specified control and target qubits.
    """
    assert i_control !=  i_target
    N = int(np.log2(len(psi)))

    if i_control < i_target:
        psi = np.reshape(psi, (2**i_control, 2, 2**(i_target-i_control-1), 2, 2**(N-i_target-1)))
        psi_out = psi.copy()
        psi_out[:, 1, :, 0, :] = psi[:, 1, :, 1, :]
        psi_out[:, 1, :, 1, :] = psi[:, 1, :, 0, :]
    else:
        psi = np.reshape(psi, (2**i_target, 2, 2**(i_control-i_target-1), 2, 2**(N-i_control-1)))
        psi_out = psi.copy()
        psi_out[:, 0, :, 1, :] = psi[:, 1, :, 1, :]
        psi_out[:, 1, :, 1, :] = psi[:, 0, :, 1, :]
    return psi_out.reshape(-1)

In [14]:
print("Apply CNOT with first qubit as control, second as target. Correct?:", np.allclose(np.kron(CNOT, np.eye(4)) @ ψtest, apply_CNOT(ψtest, 0, 1)))

Apply CNOT with first qubit as control, second as target. Correct?: True


In [15]:
print("Apply CNOT with second qubit as control, first as target. Correct?:", np.allclose(np.kron(rCNOT, np.eye(4)) @ ψtest, apply_CNOT(ψtest, 1, 0)))

Apply CNOT with second qubit as control, first as target. Correct?: True


In [16]:
# To test the implementation using the circuit from the tutorial, we first assemble the corresponding overall unitary matrix (as reference).
CNOT21 = np.kron(rCNOT, np.eye(4))
X3 = np.kron(np.eye(4), np.kron(X, np.eye(2)))
Z4 = np.kron(np.eye(8), Z)
U = Z4 @ X3 @ CNOT21 @ H1

In [17]:
# ...now we apply the gates one-by-one:
ϕ1 = apply_hadamard(ψtest, 0)
ϕ2 = apply_CNOT(ϕ1, 1, 0)
ϕ3 = apply_X(ϕ2, 2)
ϕ4 = apply_Z(ϕ3, 3)

In [18]:
print("Output state matches:", np.allclose(U @ ψtest, ϕ4))

Output state matches: True
